In [ ]:
!pip3 -qq install torch==1.1
!pip -qq install nltk==3.2.5
!pip -qq install gensim==3.6.0
!pip -qq install bokeh==1.0.4

!wget -O quora.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1ERtxpdWOgGQ3HOigqAMHTJjmOE_tWvoF"
!unzip quora.zip

import nltk
nltk.download('punkt')

In [ ]:
import time
import math
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import clear_output
%matplotlib inline

np.random.seed(42)

# Introduction to PyTorch

PyTorch is one of the most well-known frameworks for building neural networks (which is what we're gonna do in this course).

The most obvious alternative is Tensorflow, but right now (at fall of 2018) it's much less user-friendly so we'll stick to pytorch.

And come on, if you learn one of them, you'll be able to learn another.

## Automatic Differentiation

Let's start with one of the fundamental pytorch concepts - automatic differentiation (autograd).

### Computational Graphs

Computational graphs provide a very convenient way to represent functions and calculate their gradients.

For instance,
$$f = (x + y) \cdot z$$

Can be represented with this graph:  
![graph](https://github.com/DanAnastasyev/DeepNLP-Course/raw/master/Week%2003/Images/Circuit.png)  
*From [Backpropagation, Intuitions - CS231n](http://cs231n.github.io/optimization-2/)*

*The forward pass* computes value of the function (green numbers). It starts from the inputs (on the left) and applies the sequence of functions.

*The backward pass* (or *back propagation*) is designed to compute gradients of the function. That is $\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}, \frac{\partial f}{\partial z}$ in our case. It starts from the output and applies *chain rule* to compute them.

For instance, for $f = q \cdot z$, we have $\frac{\partial f}{\partial q} = z$ and $\frac{\partial f}{\partial z} = q$.  
For $q = x + y$, we have $\frac{\partial q}{\partial x} = \frac{\partial q}{\partial y} = 1$.  
Finally, we can apply the chain rule: $\frac{\partial f}{\partial x} = \frac{\partial f}{\partial q} \frac{\partial q}{\partial x} = z$.

*If you had problems with understanding the stuff above (and even if didn't), check this great tutorial: [Backpropagation, Intuitions - CS231n](http://cs231n.github.io/optimization-2/).*

Well, such calculations in pytorch are fairly simple. You just have to describe your function as a sequence of operations, like:

In [ ]:
x = torch.tensor(-2., requires_grad=True)
y = torch.tensor(5., requires_grad=True)
z = torch.tensor(-4., requires_grad=True)

q = x + y
f = q * z

Call it with some arguments and then ask it like, "Hey, calc your grads, please". And the magic happens:

![graph](https://raw.githubusercontent.com/pytorch/pytorch/master/docs/source/_static/img/dynamic_graph.gif)  
*From [github.com/pytorch/pytorch](https://github.com/pytorch/pytorch)*

Pytorch builds graph and performs backward pass - all by itself.

If you already know tensorflow, you'll see the main difference: graph is built dynamically, it hasn't be compiled and stuff.

Basically, it means that you can debug your code much more easily.

In [ ]:
f.backward()

print('df/dz =', z.grad)
print('df/dx =', x.grad)
print('df/dy =', y.grad)

The call of the `backward()` method calculates gradients for all tensors in graph (except the subgraphs where `requires_grad == False`).

*Read about autograd in pytorch in depth here: [Autograd mechanics](https://pytorch.org/docs/stable/notes/autograd.html).*

Well, the nicest thing about pytorch is that you can use it like you used numpy. You use `ndarray` all the time, right.

There is an analog for it - `tensor`. We just created few of them actually.

It stores data, like `ndarray`:

In [ ]:
x.data

And gradient (unlike `ndarray`):

In [ ]:
x.grad

Add function used to compute the gradient:

In [ ]:
q.grad_fn

And lots of meta-info:

In [ ]:
x.type(), x.shape, x.device, x.layout

Check this tutorial to learn more about tensors: [Deep Learning with PyTorch: A 60 Minute Blitz > What is PyTorch?](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py)

Sometimes we don't want to compute the gradients. To handle this cases (we'll discuss particular examples very soon), you can use context managers ([Locally disabling gradient computation](https://pytorch.org/docs/stable/autograd.html#locally-disabling-gradient-computation)):
```python
torch.autograd.no_grad()
torch.autograd.enable_grad()
torch.autograd.set_grad_enabled(mode)
```

In [ ]:
with torch.autograd.no_grad():
    x = torch.tensor(-2., requires_grad=True)
    y = torch.tensor(5., requires_grad=True)
    q = x + y

z = torch.tensor(-4., requires_grad=True)
f = q * z

f.backward()

print('df/dz =', z.grad)
print('df/dx =', x.grad)
print('df/dy =', y.grad)

Well, the question is why on earth you would want it to use :)

### Warm-up Task

To understand it, let's write a simple linear regression implementation:

In [ ]:
w_orig, b_orig = 2.6, -0.4

X = np.random.rand(100) * 10. - 5.
y_orig = w_orig * X + b_orig

y = y_orig + np.random.randn(100)

plt.plot(X, y, '.')
plt.plot(X, y_orig)
plt.show()

There are two parameters $w$ and $b$. We want them to be as near to $w_{orig}, b_{orig}$ as possible.

What are we going to optimize? Let's optimize MSE loss:
$$J(w, b) = \frac{1}{N} \sum_{i=1}^N || \hat y_i - y_i(w, b)||^2 =\frac{1}{N} \sum_{i=1}^N || \hat y_i - (w \cdot x_i + b)||^2. $$

We can use *gradient descent* algorithm to optimize it (not even stohastic right now):
$$w_{t+1} := w_t - \alpha \cdot \frac{\partial J}{\partial w}(w_t, b_t)$$
$$b_{t+1} := w_t - \alpha \cdot \frac{\partial J}{\partial b}(w_t, b_t)$$

*You see, it would be nice to use backpropagation here.*

**Task** Implement the optimization using pure numpy.

You'll need:
1. Perform the forward pass: $y(w, b) = w \cdot x + b$;
2. Find the gradients $\frac{\partial J}{\partial w}, \frac{\partial J}{\partial b}$ using backward pass;
3. Move $w, b$ in the anti-gradients direction.

In [ ]:
def display_progress(epoch, loss, w, b, X, y, y_pred):
    clear_output(True)
    print('Epoch = {}, Loss = {}, w = {}, b = {}'.format(epoch, loss, w, b))
    plt.plot(X, y, '.')
    plt.plot(X, y_pred)
    plt.show()
    time.sleep(1)


w = np.random.randn()
b = np.random.randn()

alpha = 0.01

for i in range(100):
    <calculate model's output>

    <calculate loss>

    <calculate gradients>

    <update w and b>

    if (i + 1) % 5 == 0:
        display_progress(i + 1, loss, w, b, X, y, y_pred)

assert np.abs(w - w_orig) < 0.1, 'Something went wrong :('

It's much simpler to implement the same thing using pytorch.

The forward pass will look almost the same. And we've already learnt how to perform the backward pass! Just call `loss.backward()`.

But there are a number of caveats you have to know about.

First of all, one doen't simply update `w` and `b`. Try this:

In [ ]:
w = torch.randn(1, requires_grad=True)

w -= 1.

It should fail with a message like:  
`RuntimeError: a leaf Variable that requires grad has been used in an in-place operation.`

The issue is in the support of in-place operations in autograd: [In place operations with autograd](https://pytorch.org/docs/stable/notes/autograd.html#in-place-operations-with-autograd).

But actually we are not going to perform an operation that requires gradients. We're just updating the value of the note.

To fight this problem, we can either use `no_grad` context or update the underline data used by the tensor:

In [ ]:
w.data -= 1.

Another thing you should be aware of is that the gradients are accumulating by default. So you have to update them yourself between `loss.backward()` calls:
```python
w.grad.zero_()
b.grad.zero_()
```

**Task** Implement the linear regression on pytorch.

In [ ]:
X = torch.as_tensor(X).float()
y = torch.as_tensor(y).float()

w = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

for i in range(100):
    <copy forward pass and add backward pass + parameters updates>

    if (i + 1) % 5 == 0:
        display_progress(i + 1, loss, w.item(), b.item(),
                         X.data.numpy(), y.data.numpy(), y_pred.data.numpy())

Much simpler, isn't it? :)

## Word Embeddings (via High-Level PyTorch API)

Let's move now to more high-level API, where all the good neural network parts are implemented. Quite comprehensive description of it is given in this tutorial: [What is torch.nn really?](https://pytorch.org/tutorials/beginner/nn_tutorial.html)

Last time we used gensim to train word2vec model. Now we're ready to implement our own model.

Well, almost ready. We still haven't discussed what word2vec is.

The key idea is simple: you can understand the meaning of the word by his neighbours (the words that appear frequently in its context):  
![contexts](https://image.ibb.co/mnQ2uz/2018_09_17_21_07_08.png)
*From [cs224n, Lecture 2](http://web.stanford.edu/class/cs224n/lectures/lecture2.pdf)*

The first idea is just to use counts of the words in context as a meaningful word vector.

For instance, for such simple corpus:

```
The red fox jumped
The brown fox jumped
```

we'll have following count vectors:
```
        the fox jumped red brown
red   = (1   1    1     0    0)
brown = (1   1    1     0    0)
```

You see, `red` and `brown` have similar vector! The problem is almost solved. But we have to obtain much smaller embedding vectors.

And here is what word2vec algorithms do. They build embedding vectors based on the neighbours of the word in corpus.

A nice introduction is given in this post: [king - man + woman is queen; but why?](http://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html)

Let's do some preparation work before moving to the interesting stuff.

**Task** Tokenize and lower-case texts.

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize

quora_data = pd.read_csv('train.csv')

quora_data.question1 = quora_data.question1.replace(np.nan, '', regex=True)
quora_data.question2 = quora_data.question2.replace(np.nan, '', regex=True)

texts = list(pd.concat([quora_data.question1, quora_data.question2]).unique())

tokenized_texts = [<do it there>]

assert len(tokenized_texts) == len(texts)
assert isinstance(tokenized_texts[0], list)
assert isinstance(tokenized_texts[0][0], str)

Collect the indices of the most frequent words:

In [ ]:
from collections import Counter

MIN_COUNT = 5

words_counter = Counter(token for tokens in tokenized_texts for token in tokens)
word2index = {
    '<unk>': 0
}

for word, count in words_counter.most_common():
    if count < MIN_COUNT:
        break

    word2index[word] = len(word2index)

index2word = [word for word, _ in sorted(word2index.items(), key=lambda x: x[1])]

print('Vocabulary size:', len(word2index))
print('Tokens count:', sum(len(tokens) for tokens in tokenized_texts))
print('Unknown tokens appeared:', sum(1 for tokens in tokenized_texts for token in tokens if token not in word2index))
print('Most freq words:', index2word[1:21])

### Skip-Gram Word2vec

Word2vec is actually a set of models used to build word embeddings.

We are going to start with the *skip-gram model*.

It's a very simple neural network with just two layers. It aims to build word vectors that encode information about the co-occurring words:  
![](https://i.ibb.co/nL0LLD2/Word2vec-Example.jpg)  
*From cs224n, Lecture 2*

More precisely, it models the probabilities $\{P(w_{c+j}|w_c):  j = c-k, ..., c+k, j \neq c\}$, where $k$ is the context window size, $c$ is index of the central word (which embedding we are trying to optimize).

The learnable parameters of the model are following: matrix $U$ (embeddings' matrix that is used in all downstream tasks. In gensim it's called `syn0`) and matrix $V$ - output layer of the model (in gensim it's called `syn1`).

Two vectors correspond to each word: a row in $U$ and a column in $V$. That is $U \in \mathbb{R}^{|V|, d}$ and $V \in \mathbb{R}^{d, |V|}$, where $d$ is embedding size and $|V|$ is the vocabulary size.

As a result, the neural network looks this way:  
![skip-gram](https://i.ibb.co/F54XzDC/SkipGram.png)

What's going on and how it is connected to probability and word context?

Well, the word is mapped to its embedding $u_c$. Then this embedding is multiplied to matrix $V$.

As a result, we obtain the set of scores $\{v_j^T u_c : j \in {0, \ldots, |V|}\}$. Each corresponds to the similarity between the word $w_j$ vector and our word vector. It's very similar to the cosine similarity we calculated in the previous lesson, but without normalization.

This similarities show how likely $w_j$ can be in context of word $w_c$. That means, that they can be converted to probability using the softmax function:
$$P(w_j | w_c) = \frac{\exp(v_{j}^T u_c)}{\sum_{i=1}^{|V|} \exp(v_i^T u_c)}.$$

So for each word we calculate such probability distribution over our vocabulary. It's shown in using blue bars in the picture above. More likely word - bluer is the corresponding cell.

The model learns to distribute the probabilities between the co-occuring words for the given one. We'll use cross-entropy loss for it:
$$-\sum_{-k \leq j \leq k, j \neq 0} \log \frac{\exp(v_{c+j}^T u_c)}{\sum_{i=1}^{|V|} \exp(v_i^T u_c)} \to \min_{U, V}.$$

For instance, for the sample from the picture model will be penalized if it outputs a low probability of word `over`.

Please, notice that we calculate the similarity between vectors from different vector spaces. $u_c$ is the vector from the input embeddings and $v_j$ is the vector from the output embeddings. A high similarity between them means that they co-occur frequently, not that they are similar in the syntactic role or their semantics.

On the other hand, the similarity between $u_k$ and $u_m$ means that their output distributions are similar. And that means exactly that the similarity of the count vectors we discussed before and also most probably means their syntactic or semantic similarity.

Check this demo to understand what's going on in more depth: [https://ronxin.github.io/wevi/](https://ronxin.github.io/wevi/).

Let's implement it now!

#### Batches Generations

First of all, we need to collect all the contexts from our corpus.

In [ ]:
def build_contexts(tokenized_texts, window_size):
    contexts = []
    for tokens in tokenized_texts:
        for i in range(len(tokens)):
            central_word = tokens[i]
            context = [tokens[i + delta] for delta in range(-window_size, window_size + 1)
                       if delta != 0 and i + delta >= 0 and i + delta < len(tokens)]

            contexts.append((central_word, context))

    return contexts

In [ ]:
contexts = build_contexts(tokenized_texts, window_size=2)

Check, what you got:

In [ ]:
contexts[:5]

Let's convert words to indices:

In [ ]:
contexts = [(word2index.get(central_word, 0), [word2index.get(word, 0) for word in context])
            for central_word, context in contexts]

Neural networks are optimized using stochastic gradient descent methods. Which means, we need a batch generator - a function that generates samples to optimize neural network with.

A simple batch generator looks this way:

In [ ]:
import random

def make_skip_gram_batches_iter(contexts, window_size, num_skips, batch_size):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * window_size

    central_words = [word for word, context in contexts if len(context) == 2 * window_size and word != 0]
    contexts = [context for word, context in contexts if len(context) == 2 * window_size and word != 0]

    batch_size = int(batch_size / num_skips)
    batches_count = int(math.ceil(len(contexts) / batch_size))

    print('Initializing batches generator with {} batches per epoch'.format(batches_count))

    while True:
        indices = np.arange(len(contexts))
        np.random.shuffle(indices)

        for i in range(batches_count):
            batch_begin, batch_end = i * batch_size, min((i + 1) * batch_size, len(contexts))
            batch_indices = indices[batch_begin: batch_end]

            batch_data, batch_labels = [], []

            for data_ind in batch_indices:
                central_word, context = central_words[data_ind], contexts[data_ind]

                words_to_use = random.sample(context, num_skips)
                batch_data.extend([central_word] * num_skips)
                batch_labels.extend(words_to_use)

            yield {
                'tokens': torch.cuda.LongTensor(batch_data),
                'labels': torch.cuda.LongTensor(batch_labels)
            }

Check it:

In [ ]:
batch = next(make_skip_gram_batches_iter(contexts, window_size=2, num_skips=2, batch_size=32))

batch

#### nn.Sequential

The simplest way to implement a model on pytorch is to use `nn.Sequential`. Just define the order of layers and it will apply them sequentially.

In [ ]:
model = nn.Sequential(
    nn.Embedding(len(word2index), 32),
    nn.Linear(32, len(word2index))
)

Yep, we've just defined our skip-gram model!

The construction above says that we need a `nn.Embedding` layer (a layer that maps index to a vector. In our case it would be an index from the `range(len(word2index))` and a 32-dimensional vector) following by a `nn.Linear` layer (just a dot-product of an input vector to the learnable matrix with addition of a learnable bias).

There is another pytorch's feature we haven't discussed yet. It's computations on GPU. Neural networks are usually trained on GPUs because it's much faster.

It's extremely easy to ask pytorch to perform calculations on the GPU. Just call:

In [ ]:
model.cuda()

or

In [ ]:
device = torch.device("cuda")

model = model.to(device)

We'll apply it to the data in the batch:

In [ ]:
tokens, labels = batch['tokens'], batch['labels']

Now, to perform the forward pass just call the model with its input:

In [ ]:
logits = model(batch)

We can calculate the loss now:

In [ ]:
criterion = nn.CrossEntropyLoss()

loss = criterion(logits, labels)

And, of course, perform the backward pass:

In [ ]:
loss.backward()

Finally, we have to update the parameters. We can do it as before (like, `w.data -= alpha * w.grad`). But pytorch contains predefined optimizers that can do the same things (and more complicated stuff).

We are going to use Adam optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

To update the weights just call `optimizer.step()`:

In [ ]:
print(model[1].weight)

optimizer.step()

print(model[1].weight)

And finally, don't forget to zero grads!

In [ ]:
optimizer.zero_grad()

You can ask optimizer to do it for you, you see.

#### Train Cycle

We are ready to implement the training cycle - just like we did for our linear regression.

**Task** Implement the cycle. Train the model for a few epochs.

In [ ]:
loss_every_nsteps = 1000
total_loss = 0
start_time = time.time()

for step, (batch, labels) in enumerate(make_skip_gram_batches_iter(contexts, window_size=2, num_skips=4, batch_size=128)):
    <1. convert data to tensors>

    <2. make forward pass>

    <3. make backward pass>

    <4. apply optimizer>

    <5. zero grads>

    total_loss += loss.item()

    if step != 0 and step % loss_every_nsteps == 0:
        print("Step = {}, Avg Loss = {:.4f}, Time = {:.2f}s".format(step, total_loss / loss_every_nsteps,
                                                                    time.time() - start_time))
        total_loss = 0
        start_time = time.time()

#### Analysis

To get the embedding matrix, cast the following magic:

In [ ]:
embeddings = model[0].weight.data.cpu().numpy()

That is, we get the weights from the first layer of the model, move them from gpu to cpu and convert to numpy array.

Let's check how adequate are similarities that the model learnt.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def most_similar(embeddings, index2word, word2index, word):
    word_emb = embeddings[word2index[word]]

    similarities = cosine_similarity([word_emb], embeddings)[0]
    top10 = np.argsort(similarities)[-10:]

    return [index2word[index] for index in reversed(top10)]

most_similar(embeddings, index2word, word2index, 'warm')

And visualizations!

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

from sklearn.manifold import TSNE
from sklearn.preprocessing import scale


def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    output_notebook()

    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig


def get_tsne_projection(word_vectors):
    tsne = TSNE(n_components=2, verbose=100)
    return scale(tsne.fit_transform(word_vectors))


def visualize_embeddings(embeddings, index2word, word_count):
    word_vectors = embeddings[1: word_count + 1]
    words = index2word[1: word_count + 1]

    word_tsne = get_tsne_projection(word_vectors)
    draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)


visualize_embeddings(embeddings, index2word, 100)

### Continuous Bag of Words (CBoW)

Here is an alternative word2vec model:

![](https://i.ibb.co/StXTMFH/CBOW.png)

Now we have to predict the word by its context. The context is represented as a sum of context vectors.

**Task** Implement the batch generator. It should output a context matrix `(samples_count, 2 * window_size)` which contains the context word indices and a target matrix `(samples_count)` with the central word indices.

In [ ]:
def make_cbow_batches_iter(contexts, window_size, batch_size):
    data = np.array([context for word, context in contexts if len(context) == 2 * window_size and word != 0])
    labels = np.array([word for word, context in contexts if len(context) == 2 * window_size and word != 0])

    batches_count = int(math.ceil(len(data) / batch_size))

    print('Initializing batches generator with {} batches per epoch'.format(batches_count))

    while True:
        indices = np.arange(len(data))
        np.random.shuffle(indices)

        for i in range(batches_count):
            <implement the generator>

Check it:

In [ ]:
window_size = 2
batch_size = 32

batch = next(make_cbow_batches_iter(contexts, window_size=window_size, batch_size=batch_size))

assert isinstance(batch, dict)
assert 'labels' in batch and 'tokens' in batch

assert isinstance(batch['tokens'], torch.cuda.LongTensor)
assert isinstance(batch['labels'], torch.cuda.LongTensor)

assert batch['tokens'].shape == (batch_size, 2 * window_size)
assert batch['labels'].shape == (batch_size,)

The alternative way to define a model is to inherit it from `nn.Module`. It's a more flexible approach than defining a `nn.Sequential` model so we are going to use it mostly

```python
class MyNetModel(nn.Module):
    def __init__(self, *args, **kwargs):
        super(MyNetModel, self).__init__()
        <initialize layers>
        
    def forward(self, inputs):
        <apply layers>
        return final_output
```

You have to create all the learnable parameters (usually - layers) of the model in the `__init__` method and you have to apply them in the `forward` method.

**Task** Build the `CBoWModel`.

In [ ]:
class CBoWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()

        self._embeddings = nn.Embedding(vocab_size, embedding_dim)
        self._out_layer = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        <apply the layers>

Check it:

In [ ]:
model = CBoWModel(vocab_size=len(word2index), embedding_dim=32).cuda()

outputs = model(batch['tokens'])

assert isinstance(outputs, torch.cuda.FloatTensor)
assert outputs.shape == (batch_size, len(word2index))

**Task** Train the model in the same way as before.

In [ ]:
model = CBoWModel(vocab_size=len(word2index), embedding_dim=32).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

loss_every_nsteps = 1000
total_loss = 0
start_time = time.time()

for step, batch in enumerate(make_cbow_batches_iter(contexts, window_size=2, batch_size=128)):
    <copy-paste the learning cycle>

    total_loss += loss.item()

    if step != 0 and step % loss_every_nsteps == 0:
        print("Step = {}, Avg Loss = {:.4f}, Time = {:.2f}s".format(step, total_loss / loss_every_nsteps,
                                                                    time.time() - start_time))
        total_loss = 0
        start_time = time.time()

Let's visualize what we got.

In [ ]:
visualize_embeddings(model.embeddings.weight.data.cpu().numpy(), index2word, 1000)

### Negative Sampling

What is the most computationally hard part of the model optimization? It's computation of softmax function over the vocabulary.

To improve the model's performance *negative sampling* can be used.

The idea is fairly obvious: let's predict the probability that the word $w$ can be in the context $c$: $P(D=1|w,c)$.

The probability (again!) would be a function of the similarity between vectors:
$$P(D=1|w, c) = \sigma(v_w^T u_c) = \frac 1 {1 + \exp(-v^T_w u_c)}.$$

The sigmoid function just maps the similarity to [0, 1] range.

Well, we have positive samples (word-context pairs from the corpus), but we need some negative samples too to train our model. And we can generate them!  
![Negative Sampling](https://i.ibb.co/D7rTdbr/Negative-Sampling.png)

Just sample random word instead of the correct one and hope that they don't fit the context too well.

The loss function (in CBoW setup) will be following:
$$-\log \sigma(v_c^T u_c) - \sum_{k=1}^K \log \sigma(-\tilde v_k^T u_c),$$
where $v_c$ - central word vector, $u_c$ - context vector (sum of vectors from the context), $\tilde v_1, \ldots, \tilde v_K$ - vectors of the negative samples.

Compare it with ordinary CBoW:
$$-v_c^T u_c + \log \sum_{i=1}^{|V|} \exp(v_i^T u_c).$$

You see, it's quite similar, but the sum is over a much smaller number of samples.

The sampling is performed from $U^{3/4}$, where $U$ - unigram distribution (word frequencies).

We've already calculated the word counts (they were obtained when we called `Counter(words)`).

Just convert them to frequencies and raise them to the power of $\frac 3 4$. Why exactly $\frac 3 4$? It's just a good constant, but the intuition is following:

$$P(\text{is}) = 0.9, \ P(\text{is})^{3/4} = 0.92$$
$$P(\text{Constitution}) = 0.09, \ P(\text{Constitution})^{3/4} = 0.16$$
$$P(\text{bombastic}) = 0.01, \ P(\text{bombastic})^{3/4} = 0.032$$

The probability of high-frequent words stayed the same (approximately), while low-frequent words now are more likely.

Nice description of this algorithm can be found in [cs224n lecture notes](https://github.com/maxim5/cs224n-winter-2017/blob/master/lecture_notes/cs224n-2017-notes1.pdf).

**Task** Implement Negative Sampling.

Define distribution first:

In [ ]:
words_sum_count = sum(words_counter.values())
word_distribution = np.array([(words_counter[word] / words_sum_count) ** (3 / 4) for word in index2word])
word_distribution /= word_distribution.sum()

indices = np.arange(len(word_distribution))

np.random.choice(indices, p=word_distribution, size=(32, 5))

In [ ]:
class NegativeSamplingModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()

        self._embeddings = nn.Embedding(vocab_size, embedding_dim)
        self._out_layer = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, inputs, targets, num_samples):
        '''
        inputs: (batch_size, context_size)
        targets: (batch_size)
        num_samples: int
        Returns loss calculated like in the formula above
        '''

        <calc u_c (using self._embedding) & v_c (using self._out_layer)>

        <obtain negative sample indices with shape (inputs.shape[0], num_samples) using np.random.choice>

        <calculate v_tilde - embeddings of the negative samples (using self._out_layer)>

        <calculate logsigmoid outputs (use F.logsigmoid)>

        <return mean loss>

Train and visualize the model.

In [ ]:
model = NegativeSamplingModel(vocab_size=len(word2index), embedding_dim=32).cuda()

optimizer = optim.Adam(model.parameters(), lr=0.01)

negative_samples = 20
loss_every_nsteps = 1000
total_loss = 0
start_time = time.time()

for step, batch in enumerate(make_cbow_batches_iter(contexts, window_size=2, batch_size=128)):
    <update the learning cycle accordingly>

    total_loss += loss.item()

    if step != 0 and step % loss_every_nsteps == 0:
        print("Step = {}, Avg Loss = {:.4f}, Time = {:.2f}s".format(step, total_loss / loss_every_nsteps,
                                                                    time.time() - start_time))
        total_loss = 0
        start_time = time.time()

In [ ]:
visualize_embeddings(model.embeddings.weight.data.cpu().numpy(), index2word, 1000)

### Structured Word2Vec

In the paper [Two/Too Simple Adaptations of Word2Vec for Syntax Problems (2015), Ling, Wang, et al.](https://www.aclweb.org/anthology/N/N15/N15-1142.pdf)
two ways of improvement of the embeddings are discussed: *Structured Skip-gram Model* and *Continuous Window Model*:   
![](https://i.ibb.co/56w8MC8/Structured-Word2vec.png)  
*From Two/Too Simple Adaptations of Word2Vec for Syntax Problems*

In contract to the classic word2vec, each context word has its own embedding matrix. The idea is that the words order is meaningful and by learning the order embeddings learn syntax better.

The disadvantage of such approach is that you have to learn much more parameters in the model.

**Task** Read the paper and implement at least one of the described methods.

In [ ]:
class CWindowModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, window_size):
        super().__init__()

        <create layers>

    def forward(self, inputs):
        <apply 'em>

In [ ]:
model = CWindowModel(vocab_size=len(word2index), embedding_dim=32, window_size=2).cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)

loss_every_nsteps = 1000
total_loss = 0
start_time = time.time()

for step, batch in enumerate(make_cbow_batches_iter(contexts, window_size=2, batch_size=128)):
    <copy-paste the cycle yet again>

    total_loss += loss.item()

    if step != 0 and step % loss_every_nsteps == 0:
        print("Step = {}, Avg Loss = {:.4f}, Time = {:.2f}s".format(step, total_loss / loss_every_nsteps,
                                                                    time.time() - start_time))
        total_loss = 0
        start_time = time.time()

# Supplementary Materials

## To read
### Blogs
[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[On word embeddings - Part 2: Approximating the Softmax, Sebastian Ruder](http://ruder.io/word-embeddings-softmax/index.html)  
[Word2Vec Tutorial - The Skip-Gram Model, Chris McCormick](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)  
[Word2Vec Tutorial Part 2 - Negative Sampling, Chris McCormick](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)

### Papers
[Word2vec Parameter Learning Explained (2014), Xin Rong](https://arxiv.org/abs/1411.2738)  
[Neural word embedding as implicit matrix factorization (2014), Levy, Omer, and Yoav Goldberg](http://u.cs.biu.ac.il/~nlp/wp-content/uploads/Neural-Word-Embeddings-as-Implicit-Matrix-Factorization-NIPS-2014.pdf)  

### Enhancing Embeddings
[Two/Too Simple Adaptations of Word2Vec for Syntax Problems (2015), Ling, Wang, et al.](https://www.aclweb.org/anthology/N/N15/N15-1142.pdf)  
[Not All Neural Embeddings are Born Equal (2014)](https://arxiv.org/pdf/1410.0718.pdf)  
[Retrofitting Word Vectors to Semantic Lexicons (2014), M. Faruqui, et al.](https://arxiv.org/pdf/1411.4166.pdf)  
[All-but-the-top: Simple and Effective Postprocessing for Word Representations (2017), Mu, et al.](https://arxiv.org/pdf/1702.01417.pdf)  

### Sentence Embeddings
[Skip-Thought Vectors (2015), Kiros, et al.](https://arxiv.org/pdf/1506.06726)  

### Backpropagation
[Backpropagation, Intuitions, cs231n + next parts in the Module 1](http://cs231n.github.io/optimization-2/)   
[Calculus on Computational Graphs: Backpropagation, Christopher Olah](http://colah.github.io/posts/2015-08-Backprop/)

## To watch
[cs224n "Lecture 2 - Word Vector Representations: word2vec"](https://www.youtube.com/watch?v=ERibwqs9p38&index=2&list=PLqdrfNEc5QnuV9RwUAhoJcoQvu4Q46Lja&t=0s)  
[cs224n "Lecture 5 - Backpropagation"](https://www.youtube.com/watch?v=isPiE-DBagM&index=5&list=PLqdrfNEc5QnuV9RwUAhoJcoQvu4Q46Lja&t=0s)   
